# Python Apprentice Lecture 2: DFS와 스택 (Depth-First Search and Stack)

이번 단원에서는 **깊이 우선 탐색(DFS, Depth-First Search)**과 스택의 관계를 배운다.

DFS는 그래프와 트리를 탐색하는 가장 기본적이고 중요한 알고리즘 중 하나이며, 스택 구조와 밀접한 관련이 있다.

## 학습 목표
1. DFS의 개념과 동작 원리 이해
2. DFS와 스택의 관계 파악
3. 재귀를 이용한 DFS 구현
4. 명시적 스택을 이용한 DFS 구현
5. 그래프 문제 해결에 DFS 적용
6. 연결 컴포넌트와 사이클 탐지

## 1. DFS 개념 소개

### 깊이 우선 탐색(DFS)이란?

DFS는 그래프나 트리를 탐색할 때 **한 경로를 끝까지 탐색한 후, 다른 경로를 탐색하는 방법**이다.

- **깊이(Depth) 우선**: 가능한 한 깊이 들어가서 탐색
- 막다른 길에 도달하면 되돌아가서 다른 경로 탐색
- 스택을 사용하여 구현 (재귀 또는 명시적 스택)

### DFS vs BFS 비교

```
       A
      / \
     B   C
    /   / \
   D   E   F
```

**DFS 탐색 순서**: A → B → D → C → E → F (깊이 우선)
**BFS 탐색 순서**: A → B → C → D → E → F (너비 우선)

| 구분 | DFS | BFS |
|------|-----|-----|
| 자료구조 | 스택(Stack) | 큐(Queue) |
| 탐색 방식 | 깊이 우선 | 너비 우선 |
| 메모리 사용 | 상대적으로 적음 | 상대적으로 많음 |
| 최단 경로 | 보장 안됨 | 보장됨 |
| 구현 | 재귀 또는 스택 | 큐 |

### 그래프 표현 방법

DFS를 구현하기 전에 그래프를 어떻게 표현할지 알아보자:

```
그래프 예시:
0 -- 1 -- 2
|    |    |
3 -- 4 -- 5
```

**인접 리스트(Adjacency List)** 표현:

In [ ]:
# 인접 리스트로 그래프 표현
graph = {
    0: [1, 3],
    1: [0, 2, 4],
    2: [1, 5],
    3: [0, 4],
    4: [1, 3, 5],
    5: [2, 4]
}

print("그래프 구조:")
for node, neighbors in graph.items():
    print(f"노드 {node}: {neighbors}")

## 2. DFS와 스택의 관계 손연습

위의 그래프에서 노드 0부터 시작하여 DFS를 손으로 해보자.

### 탐색 과정

```
초기 상태:
스택: [0]  (시작 노드)
방문: []   (아직 방문한 노드 없음)
```

**단계별 진행:**

| 단계 | 스택 | 현재 노드 | 이웃 노드 | 방문 완료 | 설명 |
|------|------|-----------|-----------|-----------|------|
| 1 | `[0]` | - | - | `[]` | 시작 |
| 2 | `[1, 3]` | 0 | [1, 3] | `[0]` | 노드 0 처리, 이웃들 스택에 추가 |
| 3 | `[1]` | 3 | [0, 4] | `[0, 3]` | 노드 3 처리, 0은 이미 방문, 4 추가 |
| 4 | `[1, 4]` | - | - | `[0, 3]` | 스택에서 4를 가져옴 |
| 5 | `[1]` | 4 | [1, 3, 5] | `[0, 3, 4]` | 노드 4 처리, 5만 새로 추가 |
| 6 | `[1, 5]` | - | - | `[0, 3, 4]` | 스택에서 5를 가져옴 |
| 7 | `[1]` | 5 | [2, 4] | `[0, 3, 4, 5]` | 노드 5 처리, 2만 새로 추가 |
| 8 | `[1, 2]` | - | - | `[0, 3, 4, 5]` | 스택에서 2를 가져옴 |
| 9 | `[1]` | 2 | [1, 5] | `[0, 3, 4, 5, 2]` | 노드 2 처리, 모두 방문됨 |
| 10 | `[]` | 1 | [0, 2, 4] | `[0, 3, 4, 5, 2, 1]` | 노드 1 처리, 모두 방문됨 |

**최종 탐색 순서**: 0 → 3 → 4 → 5 → 2 → 1

### 핵심 관찰

1. **스택의 LIFO 특성**: 가장 최근에 발견한 노드부터 탐색
2. **깊이 우선**: 한 방향으로 계속 들어가다가 막히면 되돌아옴
3. **방문 체크 필수**: 무한 루프 방지
4. **되돌아가기**: 스택이 자연스럽게 이전 경로로 되돌려줌

## 3. 재귀로 DFS 구현

재귀 함수는 **호출 스택(Call Stack)**을 사용하므로, 자연스럽게 스택 구조를 활용한다.

In [ ]:
def dfs_recursive(graph, start, visited=None, path=None):
    """
    재귀를 이용한 DFS 구현
    
    Args:
        graph: 인접 리스트로 표현된 그래프
        start: 시작 노드
        visited: 방문한 노드들을 저장하는 집합
        path: 탐색 경로를 저장하는 리스트
    
    Returns:
        path: 탐색한 노드들의 순서
    """
    if visited is None:
        visited = set()
    if path is None:
        path = []
    
    # 현재 노드를 방문 처리
    visited.add(start)
    path.append(start)
    print(f"방문: {start}, 현재 경로: {path}")
    
    # 인접한 모든 노드에 대해 재귀 호출
    for neighbor in graph[start]:
        if neighbor not in visited:
            print(f"  {start} → {neighbor} 탐색")
            dfs_recursive(graph, neighbor, visited, path)
        else:
            print(f"  {start} → {neighbor} 이미 방문됨")
    
    return path

In [ ]:
# 재귀 DFS 테스트
graph = {
    0: [1, 3],
    1: [0, 2, 4],
    2: [1, 5],
    3: [0, 4],
    4: [1, 3, 5],
    5: [2, 4]
}

print("=== 재귀 DFS 실행 ===")
result = dfs_recursive(graph, 0)
print(f"\n최종 탐색 순서: {result}")

### 트리 순회 예제

DFS는 트리 순회에도 사용된다:

In [ ]:
# 이진 트리 노드 클래스
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
    
    def __str__(self):
        return str(self.val)

# 전위 순회 (Pre-order): 노드 → 왼쪽 → 오른쪽
def preorder_dfs(root, result=None):
    if result is None:
        result = []
    
    if root:
        result.append(root.val)  # 현재 노드 처리
        preorder_dfs(root.left, result)   # 왼쪽 서브트리
        preorder_dfs(root.right, result)  # 오른쪽 서브트리
    
    return result

# 중위 순회 (In-order): 왼쪽 → 노드 → 오른쪽
def inorder_dfs(root, result=None):
    if result is None:
        result = []
    
    if root:
        inorder_dfs(root.left, result)   # 왼쪽 서브트리
        result.append(root.val)  # 현재 노드 처리
        inorder_dfs(root.right, result)  # 오른쪽 서브트리
    
    return result

# 후위 순회 (Post-order): 왼쪽 → 오른쪽 → 노드
def postorder_dfs(root, result=None):
    if result is None:
        result = []
    
    if root:
        postorder_dfs(root.left, result)   # 왼쪽 서브트리
        postorder_dfs(root.right, result)  # 오른쪽 서브트리
        result.append(root.val)  # 현재 노드 처리
    
    return result

In [ ]:
# 트리 구성:
#       1
#      / \
#     2   3
#    / \
#   4   5

root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

print("트리 순회 결과:")
print(f"전위 순회: {preorder_dfs(root)}")
print(f"중위 순회: {inorder_dfs(root)}")
print(f"후위 순회: {postorder_dfs(root)}")

## 4. 명시적 스택으로 DFS 구현

재귀 대신 스택을 직접 사용해서 DFS를 구현해보자. Lecture 1에서 만든 Stack 클래스를 재사용한다.

In [ ]:
# Lecture 1에서 만든 Stack 클래스 (재사용)
class Stack:
    def __init__(self):
        self.items = []
    
    def push(self, item):
        self.items.append(item)
    
    def pop(self):
        if self.is_empty():
            raise IndexError("Stack is empty")
        return self.items.pop()
    
    def peek(self):
        if self.is_empty():
            raise IndexError("Stack is empty")
        return self.items[-1]
    
    def is_empty(self):
        return len(self.items) == 0
    
    def size(self):
        return len(self.items)
    
    def __str__(self):
        return f"Stack: {self.items}"

In [ ]:
def dfs_iterative(graph, start):
    """
    명시적 스택을 이용한 DFS 구현
    
    Args:
        graph: 인접 리스트로 표현된 그래프
        start: 시작 노드
    
    Returns:
        path: 탐색한 노드들의 순서
    """
    visited = set()
    path = []
    stack = Stack()
    
    # 시작 노드를 스택에 추가
    stack.push(start)
    print(f"초기화: {stack}")
    
    while not stack.is_empty():
        # 스택에서 노드를 꺼냄
        current = stack.pop()
        print(f"\n스택에서 꺼냄: {current}, 남은 스택: {stack}")
        
        # 이미 방문한 노드면 스킵
        if current in visited:
            print(f"  {current}은 이미 방문됨")
            continue
        
        # 현재 노드 방문 처리
        visited.add(current)
        path.append(current)
        print(f"  {current} 방문 처리, 현재 경로: {path}")
        
        # 인접한 노드들을 스택에 추가 (역순으로 추가하여 올바른 순서 보장)
        neighbors = graph[current]
        for neighbor in reversed(neighbors):  # 역순으로 추가
            if neighbor not in visited:
                stack.push(neighbor)
                print(f"    {neighbor} 스택에 추가")
        
        print(f"  업데이트된 스택: {stack}")
    
    return path

In [ ]:
# 반복적 DFS 테스트
graph = {
    0: [1, 3],
    1: [0, 2, 4],
    2: [1, 5],
    3: [0, 4],
    4: [1, 3, 5],
    5: [2, 4]
}

print("=== 반복적 DFS 실행 ===")
result = dfs_iterative(graph, 0)
print(f"\n최종 탐색 순서: {result}")

### 재귀 vs 반복 비교

| 특성 | 재귀 DFS | 반복 DFS |
|------|----------|----------|
| **구현** | 간단하고 직관적 | 약간 복잡 |
| **메모리** | 호출 스택 사용 | 명시적 스택 사용 |
| **스택 오버플로우** | 깊은 그래프에서 발생 가능 | 발생하지 않음 |
| **제어** | 제한적 | 더 세밀한 제어 가능 |
| **성능** | 함수 호출 오버헤드 | 상대적으로 빠름 |
| **디버깅** | 호출 스택 추적 어려움 | 스택 상태 확인 가능 |

In [ ]:
# 두 방법의 결과 비교
print("=== 두 구현 방법 비교 ===")
print("재귀 DFS:")
recursive_result = dfs_recursive(graph, 0)
print(f"결과: {recursive_result}")

print("\n반복 DFS:")
iterative_result = dfs_iterative(graph, 0)
print(f"결과: {iterative_result}")

print(f"\n결과가 같은가? {recursive_result == iterative_result}")

## 5. 실습 1: 그래프에서 경로 찾기

DFS를 사용해서 두 노드 사이에 경로가 존재하는지 확인하고, 경로를 찾아보자.

In [ ]:
def find_path_dfs(graph, start, target):
    """
    DFS를 사용하여 시작 노드에서 목표 노드까지의 경로 찾기
    
    Args:
        graph: 그래프
        start: 시작 노드
        target: 목표 노드
    
    Returns:
        경로가 존재하면 경로 리스트, 없으면 None
    """
    visited = set()
    path = []
    
    def dfs_helper(current):
        visited.add(current)
        path.append(current)
        print(f"방문: {current}, 현재 경로: {path}")
        
        # 목표 노드에 도달한 경우
        if current == target:
            print(f"목표 {target} 발견!")
            return True
        
        # 인접한 노드들 탐색
        for neighbor in graph[current]:
            if neighbor not in visited:
                if dfs_helper(neighbor):
                    return True
        
        # 이 경로로는 목표에 도달할 수 없음 (백트래킹)
        path.pop()
        print(f"백트래킹: {current} 제거, 현재 경로: {path}")
        return False
    
    if dfs_helper(start):
        return path.copy()
    else:
        return None

In [ ]:
# 경로 찾기 테스트
graph = {
    0: [1, 3],
    1: [0, 2, 4],
    2: [1, 5],
    3: [0, 4],
    4: [1, 3, 5],
    5: [2, 4]
}

print("=== 경로 찾기 테스트 ===")
test_cases = [
    (0, 5),  # 연결됨
    (2, 3),  # 연결됨
    (1, 4),  # 연결됨
]

for start, target in test_cases:
    print(f"\n{start} → {target} 경로 찾기:")
    path = find_path_dfs(graph, start, target)
    if path:
        print(f"경로 발견: {' → '.join(map(str, path))}")
    else:
        print("경로 없음")
    print("-" * 40)

### 연결성 확인 함수

두 노드가 연결되어 있는지 간단히 확인하는 함수:

In [ ]:
def is_connected_dfs(graph, start, target):
    """
    두 노드가 연결되어 있는지 DFS로 확인
    """
    visited = set()
    stack = Stack()
    stack.push(start)
    
    while not stack.is_empty():
        current = stack.pop()
        
        if current == target:
            return True
        
        if current in visited:
            continue
        
        visited.add(current)
        
        for neighbor in graph[current]:
            if neighbor not in visited:
                stack.push(neighbor)
    
    return False

# 연결성 테스트
print("=== 연결성 확인 테스트 ===")
test_pairs = [(0, 5), (1, 3), (2, 4)]

for start, target in test_pairs:
    connected = is_connected_dfs(graph, start, target)
    print(f"{start}과 {target}은 {'연결됨' if connected else '연결되지 않음'}")

## 6. 실습 2: 연결된 컴포넌트 찾기

그래프가 몇 개의 독립된 부분(연결 컴포넌트)으로 나뉘어 있는지 찾아보자.

```
예시:
0 -- 1    2 -- 3
|    |         |
4 -- 5         6

컴포넌트 1: [0, 1, 4, 5]
컴포넌트 2: [2, 3, 6]
```

In [ ]:
def find_connected_components(graph):
    """
    그래프의 모든 연결 컴포넌트를 찾기
    
    Args:
        graph: 그래프
    
    Returns:
        components: 각 컴포넌트의 노드들을 담은 리스트
    """
    visited = set()
    components = []
    
    def dfs_component(start):
        """하나의 컴포넌트에 속한 모든 노드 찾기"""
        component = []
        stack = Stack()
        stack.push(start)
        
        while not stack.is_empty():
            current = stack.pop()
            
            if current in visited:
                continue
            
            visited.add(current)
            component.append(current)
            print(f"  노드 {current} 추가, 현재 컴포넌트: {component}")
            
            for neighbor in graph[current]:
                if neighbor not in visited:
                    stack.push(neighbor)
        
        return sorted(component)
    
    # 모든 노드에 대해 DFS 수행
    for node in graph:
        if node not in visited:
            print(f"\n새 컴포넌트 탐색 시작: {node}")
            component = dfs_component(node)
            components.append(component)
            print(f"컴포넌트 완성: {component}")
    
    return components

In [ ]:
# 분리된 그래프 예제
disconnected_graph = {
    0: [1, 4],
    1: [0, 5],
    4: [0, 5],
    5: [1, 4],
    2: [3, 6],
    3: [2, 6],
    6: [2, 3],
    7: [8],
    8: [7]
}

print("=== 연결 컴포넌트 찾기 ===")
components = find_connected_components(disconnected_graph)

print(f"\n총 {len(components)}개의 연결 컴포넌트 발견:")
for i, component in enumerate(components, 1):
    print(f"컴포넌트 {i}: {component}")

In [ ]:
# 연결된 그래프에서도 테스트
connected_graph = {
    0: [1, 3],
    1: [0, 2, 4],
    2: [1, 5],
    3: [0, 4],
    4: [1, 3, 5],
    5: [2, 4]
}

print("\n=== 연결된 그래프 테스트 ===")
components = find_connected_components(connected_graph)

print(f"\n총 {len(components)}개의 연결 컴포넌트 발견:")
for i, component in enumerate(components, 1):
    print(f"컴포넌트 {i}: {component}")

## 7. 실습 3: 사이클 탐지

그래프에 사이클(순환)이 있는지 DFS로 확인해보자.

**사이클**: 한 노드에서 시작해서 다른 노드들을 거쳐 다시 원점으로 돌아오는 경로

In [ ]:
def has_cycle_undirected(graph):
    """
    무방향 그래프에서 사이클 탐지
    
    Args:
        graph: 무방향 그래프
    
    Returns:
        bool: 사이클이 있으면 True, 없으면 False
    """
    visited = set()
    
    def dfs_cycle_check(current, parent):
        visited.add(current)
        print(f"방문: {current} (부모: {parent})")
        
        for neighbor in graph[current]:
            if neighbor == parent:  # 부모 노드로 되돌아가는 건 사이클 아님
                continue
            
            if neighbor in visited:
                print(f"사이클 발견! {current} → {neighbor} (이미 방문됨)")
                return True
            
            if dfs_cycle_check(neighbor, current):
                return True
        
        return False
    
    # 모든 컴포넌트 확인
    for node in graph:
        if node not in visited:
            print(f"\n새 컴포넌트에서 사이클 검사 시작: {node}")
            if dfs_cycle_check(node, -1):
                return True
    
    return False

In [ ]:
# 사이클이 있는 그래프
cycle_graph = {
    0: [1, 3],
    1: [0, 2, 4],
    2: [1, 5],
    3: [0, 4],
    4: [1, 3, 5],  # 0-1-4-3-0 사이클 존재
    5: [2, 4]
}

# 사이클이 없는 그래프 (트리)
no_cycle_graph = {
    0: [1, 2],
    1: [0, 3, 4],
    2: [0, 5],
    3: [1],
    4: [1],
    5: [2]
}

print("=== 사이클 탐지 테스트 ===")

print("1. 사이클이 있는 그래프:")
has_cycle1 = has_cycle_undirected(cycle_graph)
print(f"결과: {'사이클 있음' if has_cycle1 else '사이클 없음'}")

print("\n2. 사이클이 없는 그래프 (트리):")
has_cycle2 = has_cycle_undirected(no_cycle_graph)
print(f"결과: {'사이클 있음' if has_cycle2 else '사이클 없음'}")

### 방향 그래프에서의 사이클 탐지

방향 그래프에서는 **현재 경로에서 다시 만나는 노드**를 확인해야 한다:

In [ ]:
def has_cycle_directed(graph):
    """
    방향 그래프에서 사이클 탐지
    """
    WHITE = 0  # 미방문
    GRAY = 1   # 방문 중 (현재 경로에 있음)
    BLACK = 2  # 방문 완료
    
    colors = {node: WHITE for node in graph}
    
    def dfs_directed(node):
        colors[node] = GRAY
        print(f"방문 시작: {node} (GRAY)")
        
        for neighbor in graph[node]:
            if colors[neighbor] == GRAY:
                print(f"사이클 발견! {node} → {neighbor} (현재 경로에 있음)")
                return True
            elif colors[neighbor] == WHITE:
                if dfs_directed(neighbor):
                    return True
        
        colors[node] = BLACK
        print(f"방문 완료: {node} (BLACK)")
        return False
    
    for node in graph:
        if colors[node] == WHITE:
            if dfs_directed(node):
                return True
    
    return False

# 방향 그래프 예제
directed_cycle_graph = {
    0: [1],
    1: [2],
    2: [0, 3],  # 0 → 1 → 2 → 0 사이클
    3: []
}

directed_no_cycle_graph = {
    0: [1, 2],
    1: [3],
    2: [3],
    3: []
}

print("\n=== 방향 그래프 사이클 탐지 ===")

print("1. 사이클이 있는 방향 그래프:")
has_directed_cycle1 = has_cycle_directed(directed_cycle_graph)
print(f"결과: {'사이클 있음' if has_directed_cycle1 else '사이클 없음'}")

print("\n2. 사이클이 없는 방향 그래프:")
has_directed_cycle2 = has_cycle_directed(directed_no_cycle_graph)
print(f"결과: {'사이클 있음' if has_directed_cycle2 else '사이클 없음'}")

## 🎯 학습 정리

### 핵심 내용 요약

1. **DFS의 특성**
   - 깊이 우선 탐색: 한 경로를 끝까지 탐색
   - 스택 구조 활용 (LIFO)
   - 재귀 또는 명시적 스택으로 구현 가능

2. **DFS와 스택의 관계**
   - 재귀 = 호출 스택 (암시적 스택)
   - 반복 = 명시적 스택
   - 되돌아가기(백트래킹) 자연스럽게 지원

3. **주요 응용**
   - 경로 찾기
   - 연결성 확인
   - 연결 컴포넌트 찾기
   - 사이클 탐지
   - 트리 순회

4. **시간/공간 복잡도**
   - 시간 복잡도: O(V + E) (V: 정점 수, E: 간선 수)
   - 공간 복잡도: O(V) (visited 배열 + 스택)

### DFS vs BFS 선택 기준

**DFS를 사용하는 경우:**
- 경로의 존재만 확인하면 될 때
- 모든 경로를 탐색해야 할 때
- 메모리 사용량을 줄이고 싶을 때
- 백트래킹이 필요할 때

**BFS를 사용하는 경우:**
- 최단 경로를 찾을 때
- 특정 거리 내의 모든 노드를 찾을 때

### 다음 단계

- **Lecture 3**: 미로 탐색과 스택 (DFS 실전 응용)
- **고급 주제**: 백트래킹, 위상 정렬, 강연결 컴포넌트
- **최적화**: 메모이제이션, 동적 계획법과의 결합

## 💡 연습 문제

1. **이진 트리 검증**: 주어진 트리가 이진 탐색 트리(BST)인지 DFS로 확인하기

2. **섬의 둘레**: 2D 그리드에서 섬의 둘레를 DFS로 계산하기

3. **단어 검색**: 2D 문자 보드에서 특정 단어가 존재하는지 DFS로 찾기

4. **최장 경로**: 트리에서 가장 긴 경로의 길이를 DFS로 구하기

5. **위상 정렬**: 방향 그래프를 위상 정렬하기 (DFS 응용)

이상으로 Python Apprentice Lecture 2를 마칩니다! 🚀